In [1]:
import pymongo
import cassiopeia as cass
import numpy as np
import pandas as pd
import pickle
from sklearn import svm
from sklearn import tree
from sklearn.model_selection import cross_val_score
import scipy.optimize

In [2]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["LoLData"]
mycol = mydb["modelData"]

In [3]:
scoring = mydb['match']

In [248]:
scoring.find_one({'_id': 3175062854})

{'_id': 3177363192,
 'region': 'NORTH_AMERICA',
 'participantData': {'1': {'champion': 'Kayle',
   'startingItems': ['Warding Totem (Trinket)', 'Corrupting Potion'],
   'participantId': 1,
   'frequentPosition': 'TOP',
   'experienceAt12': 5266,
   'creepScoreAt12': 88,
   'neutralKillsAt12': 0,
   'summonerSpells': ['Teleport', 'Flash']},
  '2': {'champion': 'Ziggs',
   'startingItems': ['Health Potion', "Spellthief's Edge"],
   'participantId': 2,
   'frequentPosition': 'BOT',
   'experienceAt12': 3120,
   'creepScoreAt12': 5,
   'neutralKillsAt12': 0,
   'summonerSpells': ['Ignite', 'Flash']},
  '3': {'champion': 'Irelia',
   'startingItems': ['Warding Totem (Trinket)', 'Corrupting Potion'],
   'participantId': 3,
   'frequentPosition': 'MID',
   'experienceAt12': 4934,
   'creepScoreAt12': 100,
   'neutralKillsAt12': 0,
   'summonerSpells': ['Ignite', 'Flash']},
  '4': {'champion': 'Lee Sin',
   'startingItems': ['Warding Totem (Trinket)',
    'Refillable Potion',
    "Hunter's Mac

In [7]:
STARTING_ITEMS = sorted(
("Boots of Speed","Faerie Charm","Rejuvenation Bead","Sapphire Crystal","Ruby Crystal","Cloth Armor","Null-Magic Mantle","Long Sword","Hunter's Talisman","Hunter's Machete","Dagger",
"Brawler's Gloves","Amplifying Tome","Doran's Shield","Doran's Blade","Doran's Ring","The Dark Seal","Cull","Ancient Coin","Relic Shield","Spellthief's Edge","Corrupting Potion"))

STARTING_ITEMS = {STARTING_ITEMS[i] : i for i in range(len(STARTING_ITEMS))}

SUMMONER_SPELLS = cass.get_summoner_spells("NA")
SUMMONER_SPELLS = sorted([spell.name for spell in SUMMONER_SPELLS if cass.GameMode.classic in spell.modes])
SUMMONER_SPELLS = {SUMMONER_SPELLS[i] : i for i in range(len(SUMMONER_SPELLS))}

POSITIONS = sorted(('TOP','MID','BOT','JUNGLE'))
POSITIONS = {POSITIONS[i] : i for i in range(len(POSITIONS))}
# CHAMPIONS = cass.get_champions("NA")
# CHAMPIONS = sorted([champion.name for champion in CHAMPIONS])
# CHAMPIONS = {CHAMPIONS[i] : i for i in range(len(CHAMPIONS))}

print(len(STARTING_ITEMS), len(SUMMONER_SPELLS))

22 9


In [8]:
trainingColumns = ['CS','Exp','MonsterKills'] + list(SUMMONER_SPELLS.keys()) + list(POSITIONS.keys()) + \
list(STARTING_ITEMS.keys()) + ['ParticipantId', 'GameId','VerifiedRole']
def featureGetCS(participant):
    return [participant['creepScoreAt12']]

def featureGetStartingItems(participant):
    StartingItems = [0] * len(STARTING_ITEMS)
    if 'startingItems' in participant:
        for item in participant['startingItems']:
            if item in STARTING_ITEMS:
                StartingItems[STARTING_ITEMS[item]] = 1
    return StartingItems

# def featureGetChampion(participant):
#     champions = [0] * len(CHAMPIONS)
#     for champion in CHAMPIONS:
#         if participant['champion'] == champion:
#             champions[CHAMPIONS[participant['champion']]] = 1
#     return champions

def featureGetFrequentPosition(participant):
    positions = [0] * len(POSITIONS)
    for p in POSITIONS:
        if participant['frequentPosition'] == p:
            positions[POSITIONS[participant['frequentPosition']]] = 1
    return positions

def featureGetExperience(participant):
    return [participant['experienceAt12']]

def featureGetSummonerSpells(participant):
    spells = [0] * len(SUMMONER_SPELLS)
    for spell in participant['summonerSpells']:
        if spell in SUMMONER_SPELLS:
            spells[SUMMONER_SPELLS[spell]] = 1
    return spells

def featureGetNeutralKills(participant):
    return [participant['neutralKillsAt12']]

def getFeatures(participant):
    features = {
        "cs" : featureGetCS(participant),
        "exp" : featureGetExperience(participant),
        "monsters" : featureGetNeutralKills(participant),
        "spells" : featureGetSummonerSpells(participant),
        "position" : featureGetFrequentPosition(participant),
        "items" : featureGetStartingItems(participant)
        #"champion" : featureGetChampion(participant)
    }
    '''np.array((
        *features['cs'],
        *features['exp'],
        *features['monsters'],
        *features['spells'],
        *features['position'],
        *features['items'],
        *features['champion']
    ))'''
    l = list(features.values())
    return [item for sublist in l for item in sublist]

In [9]:
def getGameData(game):
    data = []
    features = []
    gameId = game['_id']
    participants = game['participantData']
    for participant in participants.values():
        participantId = participant['participantId']
        if 'validatedRole' in game:
            verifiedRole = participant['validatedRole']
            features.append(verifiedRole)
        features = getFeatures(participant)
        data.append(features)        
    return data

def getData(data):
    rows = []
    for d in data:
        gameId = d['_id']
        participants = d['participantData']
        for participant in participants.values():
            participantId = participant['participantId']
            verifiedRole = participant['validatedRole']
            features = getFeatures(participant)
            features.append(participantId)
            features.append(gameId)
            features.append(verifiedRole)
            rows.append(features)
    return rows

In [10]:
#trainingColumns
myData = getData(mycol.find())
print(len(myData[0]), len(trainingColumns))

41 41


In [18]:
df = pd.DataFrame(getData(mycol.find()), columns=trainingColumns)

In [19]:
#y.value_counts()
print(len(df[df['VerifiedRole']=='undefined']))
print(len(df))
df.drop(df[df['VerifiedRole']=='undefined'].index, inplace=True)
print(len(df))

2
5870
5868


In [20]:
y = df['VerifiedRole']
x = df[df.columns[:-3]]
scoringColumns = list(x.columns)

In [21]:
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=10, random_state=0)
clf.fit(x, y)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [22]:
scores = cross_val_score(clf, x, y, cv=5)
print(scores)
scores *= 100
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[0.99744681 0.99489362 0.99659574 0.99744246 0.9982906 ]
Accuracy: 99.69 (+/- 0.23)


In [23]:
filename = 'finalized_model.sav'
pickle.dump(clf, open(filename, 'wb'))

In [210]:
#loaded_model = pickle.load(open(filename, 'rb'))
LABELS = ['BOTTOM', 'JUNGLE', 'MIDDLE', 'SUPPORT', 'TOP']
def split_features_by_team(features, teamsize=5):
    teams = []
    assert len(features) % teamsize == 0
    for i in range(0, len(features), teamsize):
        teams.append(features[i:i+teamsize])
    return teams

[[1. 1. 1. 1. 0. 1.]
 [1. 1. 1. 0. 1. 1.]
 [1. 1. 0. 1. 1. 1.]
 [1. 0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1. 1.]]
['TOP', 'UTILITY', 'MIDDLE', 'JUNGLE', 'BOTTOM']
[[0. 1. 1. 1. 1. 1.]
 [1. 0. 1. 1. 1. 1.]
 [1. 1. 0. 1. 1. 1.]
 [1. 1. 1. 1. 0. 1.]
 [1. 1. 1. 0. 1. 1.]]
['BOTTOM', 'JUNGLE', 'MIDDLE', 'TOP', 'UTILITY']


In [219]:
teams = split_features_by_team(gameData)

In [237]:
def predict_team(team, clf, verbose=False):
    assert len(team) == len(LABELS)
    probs = 1 - clf.predict_proba(team)
    print(probs)
    row_ind, result = scipy.optimize.linear_sum_assignment(probs)
    cost = probs[row_ind, result].sum() / len(team)
    if verbose:
        print(probs)
        print(result)
    return [LABELS[i] for i in result], cost
def get_roles(game):
    gameData = getGameData(game)
    teams = split_features_by_team(gameData)
    roleList = []
    for team in teams:
        roles, cost = predict_team(team, clf)
        roleList+=roles
    print(roleList)
    
    participants = game['participantData']
    for i in range(len(participants)):
        participants[str(i+1)]['validatedRole']=roleList[i]

In [238]:
sourceGame = scoring.find_one()
get_roles(sourceGame)

[[1. 1. 1. 1. 0. 1.]
 [1. 1. 1. 0. 1. 1.]
 [1. 1. 0. 1. 1. 1.]
 [1. 0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1. 1.]]
[[0. 1. 1. 1. 1. 1.]
 [1. 0. 1. 1. 1. 1.]
 [1. 1. 0. 1. 1. 1.]
 [1. 1. 1. 1. 0. 1.]
 [1. 1. 1. 0. 1. 1.]]
['TOP', 'UTILITY', 'MIDDLE', 'JUNGLE', 'BOTTOM', 'BOTTOM', 'JUNGLE', 'MIDDLE', 'TOP', 'UTILITY']


In [239]:
sourceGame['participantData']['1']

{'champion': 'Kayle',
 'startingItems': ['Warding Totem (Trinket)', 'Corrupting Potion'],
 'participantId': 1,
 'frequentPosition': 'TOP',
 'experienceAt12': 5266,
 'creepScoreAt12': 88,
 'neutralKillsAt12': 0,
 'summonerSpells': ['Teleport', 'Flash'],
 'validatedRole': 'TOP'}